In [6]:
import sys, os, re
import argparse
import numpy  as np
from collections import defaultdict, Iterable

import lsst.daf.persistence as dafPer
import lsst.afw.coord       as afwCoord
import lsst.afw.image       as afwImage
import lsst.afw.table       as afwTable
import lsst.afw.coord       as afwCoord
import lsst.afw.geom        as afwGeom
import lsst.pex.exceptions

In [7]:
def getVisits(tpDict, butler, filter='HSC-I'):
    """
    get visits that cover the tract/patch list

    From Claire Lackner's getTractPatches.py in gxystack
    """
    visitDict = defaultdict(set)

    for tract in tpDict:
        for patch in tpDict[tract]:
            dataId =  {'tract':tract,
                       'patch':"{0!s},{1!s}".format(*patch),
                       'filter':filter}
            coadd = butler.get("deepCoadd", dataId)
            try:
                ccds = coadd.getInfo().getCoaddInputs().ccds
            except lsst.pex.exceptions.LsstCppException:
                continue
            print dataId
            for ccd in ccds:
                visitDict[ccd.get('visit')].add(ccd.get('ccd'))
    return visitDict

In [14]:
def raDec2VisitCcd(ra, dec, root, filter='HSC-I', butler=None):
    """TODO: Docstring for raDec2VisitCcd.

    Find all the (visit, ccd) of HSC single images that cover given (RA, DEC)

    Based on Claire Lackner's getTractPatches.py in gxystack

    """

    # Get the butler if not provided
    if butler is None:
        butler = dafPer.Butler(root)
    skyMap = butler.get("deepCoadd_skyMap", immediate=True)


    # Fake a very simple source table
    schema = afwTable.SourceTable.makeMinimalSchema()
    table  = afwTable.SourceTable.make(schema)
    scat   = afwTable.SourceCatalog(table)

    # See if the input is number or array
    if isinstance(ra, (int, float)) and isinstance(dec, (int, float)):
        s = scat.addNew()
        s.setId(1)
        s.setRa(float(ra)*afwGeom.degrees)
        s.setDec(float(dec)*afwGeom.degrees)
    elif isinstance(ra, Iterable) and isinstance(dec, Iterable) and (len(ra) == len(dec)):
        raArr  = np.asarray(ra)
        decArr = np.asarray(dec)
        for i, (rr, dd) in enumerate(zip(raArr, decArr)):
            s = scat.addNew()
            s.setId(int(i))
            s.setRa(float(rr)*afwGeom.degrees)
            s.setDec(float(dd)*afwGeom.degrees)

    # Get the list of tract and patch
    tpList = [skyMap.findTractPatchList([scat[i].get("coord"),])
              for i in range(len(scat))]
    tpDict = defaultdict(set)

    #
    for tp in tpList:
        for tract in tp:
            for patch in tract[1]:
                tpDict[tract[0].getId()].add(patch.getIndex())
    print "number of patches ", sum([len(td) for td in tpDict.values()])

    visitDict = getVisits(tpDict[0], butler, filter=filter)

    return visitDict

In [48]:
rootDir = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.6.1_20150325'
ra  = 150.58048
dec = 2.29262
filter = 'HSC-I'

In [16]:
visit = raDec2VisitCcd(ra, dec, rootDir)

number of patches  1


In [20]:
print visit

defaultdict(<type 'set'>, {})


# Break Down

In [23]:
butler = dafPer.Butler(rootDir)
skyMap = butler.get("deepCoadd_skyMap", immediate=True)

In [24]:
# Fake a very simple source table
schema = afwTable.SourceTable.makeMinimalSchema()
table  = afwTable.SourceTable.make(schema)
scat   = afwTable.SourceCatalog(table)

In [25]:
s = scat.addNew()
s.setId(1)
s.setRa(float(ra)*afwGeom.degrees)
s.setDec(float(dec)*afwGeom.degrees)

In [28]:
# Get the list of tract and patch
tpList = [skyMap.findTractPatchList([scat[i].get("coord"),])
              for i in range(len(scat))]
tpDict = defaultdict(set)

In [32]:
raDec = scat[0].get("coord")

In [38]:
tpList = [skyMap.findTractPatchList([raDec, ])]
tpDict = defaultdict(set)

In [44]:
for tp in tpList:
    for tract in tp:
        for patch in tract[1]:
            tpDict[tract[0].getId()].add(patch.getIndex())
print "number of patches ", sum([len(td) for td in tpDict.values()])

number of patches  1


In [58]:
a = tpList[0]

In [60]:
t = a[0]

In [63]:
p = t[1]

In [70]:
t

(TractInfo(id=9813, ctrCoord=(-0.86752, 0.49587, 0.038936)),
 (PatchInfo(index=(2, 4), innerBBox=Box2I((8000, 16000), (11999, 19999)), outerBBox=Box2I((7900, 15900), (12099, 20099))),))

In [45]:
print tpDict.items()

[(9813, set([(2, 4)]))]


In [49]:
visitDict = defaultdict(set)

for tract in tpDict:
    for patch in tpDict[tract]:
        dataId =  {'tract':tract,
                   'patch':"{0!s},{1!s}".format(*patch),
                   'filter':filter}
        coadd = butler.get("deepCoadd", dataId)
        try:
            ccds = coadd.getInfo().getCoaddInputs().ccds
        except lsst.pex.exceptions.LsstCppException:
            continue
        print dataId
        for ccd in ccds:
            visitDict[ccd.get('visit')].add(ccd.get('ccd'))

{'filter': 'HSC-I', 'patch': '2,4', 'tract': 9813}


In [57]:
for visit, ccds in visitDict.items():
    for ccd in ccds:
        print visit, ccd

19712 55
19712 47
19712 39
24324 8
24324 2
24324 3
24324 7
19718 80
19718 73
19718 86
24332 98
24332 99
24336 8
24336 2
24336 3
24336 7
24340 98
24340 99
24344 3
24348 99
1242 39
1242 40
1242 47
1242 48
1242 55
1242 56
24352 2
24352 3
24356 98
24356 99
1248 38
1248 39
1248 46
1248 47
1248 54
1248 55
1228 55
1228 47
1228 39
19658 55
19658 47
19658 39
19660 64
19660 47
19660 48
19660 55
19660 56
19660 63
1230 64
1230 47
1230 48
1230 55
1230 56
1230 63
1232 39
1232 40
1232 47
1232 48
1232 55
1232 56
19666 3
1236 46
1236 47
1236 54
1236 55
1236 62
1236 63
19662 39
19662 40
19662 47
19662 48
19662 55
19662 56
1238 39
1238 40
1238 47
1238 48
1238 55
1238 56
1240 64
1240 47
1240 48
1240 55
1240 56
1240 63
19674 80
19674 81
19674 86
19674 87
19674 92
19674 93
1244 38
1244 39
1244 46
1244 47
1244 54
1244 55
1246 38
1246 39
1246 46
1246 47
1246 54
1246 55
19680 38
19680 39
19680 46
19680 47
19680 54
19680 55
19682 38
19682 39
19682 46
19682 47
19682 54
19682 55
19684 64
19684 47
19684 48
19684 5